In [1]:
import os
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pathlib import Path
import argparse



def calibrate_camera(imgs_path, nx, ny):
    images = glob.glob(imgs_path + "/*.jpg")
    # images = os.listdir(imgs_path)
    objpoints, imgpoints = [], []
    objp = np.zeros((nx*ny, 3), np.float32)
    objp[:, :2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2)
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

        if ret is True:
            objpoints.append(objp)
            imgpoints.append(corners)

    return objpoints, imgpoints

In [2]:
def show_image(img):
    cv2.namedWindow("img", cv2.WINDOW_NORMAL)
    cv2.imshow("img", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
objpoints, imgpoints = calibrate_camera("camera_cal/", 9, 6)

In [4]:
def get_undist_img(img, objpoints, imgpoints):
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[1::-1], None, None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

In [5]:
def perspective_transform(undist_img):
    offset = 10
    src = np.float32([[566, 457],
                  [750, 457],
                  [1220, undist_img.shape[0]],
                  [130, undist_img.shape[0]]
                 ])

    dst = np.float32([[offset, offset],
                 [undist_img.shape[1] - offset, offset],
                 [undist_img.shape[1] - offset, undist_img.shape[0] - offset],
                 [offset, undist_img.shape[0] - offset]])

    Minv = cv2.getPerspectiveTransform(dst, src)
    M = cv2.getPerspectiveTransform(src, dst)

    warped = cv2.warpPerspective(undist_img, M, undist_img.shape[1::-1], flags=cv2.INTER_LINEAR)
    return warped, M, Minv

In [6]:
img = cv2.imread("test_images_new/7.jpg")

In [71]:
show_image(img)

In [7]:
undist = get_undist_img(img, objpoints, imgpoints)

In [8]:
warped, M, Minv = perspective_transform(undist)

In [10]:
show_image(warped)

In [9]:
def abs_sobel_thresh(img, sobel_kernel=3, orient='x',  thresh=(100,255)):
    if orient=='x':
        sobel = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    if orient=='y':
        sobel = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    sobelabs = np.abs(sobel)
    scaledsobel = np.uint8(255*sobelabs/np.max(sobelabs))
    sbinary = np.zeros_like(scaledsobel)
    sbinary[(scaledsobel >= thresh[0]) & (scaledsobel <= thresh[1])] = 255
    return sbinary

In [10]:
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1)
    magnitude = np.sqrt((sobelx)**2 + (sobely)**2)
    scaledsobel = np.uint8(255*magnitude/np.max(magnitude))
    sbinary = np.zeros_like(scaledsobel)
    sbinary[(scaledsobel >= mag_thresh[0]) & (scaledsobel <= mag_thresh[1])] = 255
    return sbinary

In [11]:
def dir_threshold(img, sobel_kernel=3, thresh=(0.7, 1.3)):
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    abs_sobelx = np.abs(sobelx)
    abs_sobely = np.abs(sobely)
    direction = np.arctan2(abs_sobely, abs_sobelx)
    binary_output = np.zeros_like(direction)
    binary_output[(direction >= thresh[0]) & (direction <= thresh[1])] = 255
    return binary_output

In [12]:
ksize = 3

In [13]:
hls = cv2.cvtColor(warped, cv2.COLOR_BGR2HLS)
s_channel = hls[:,:,2]
l_channel = hls[:,:,1]

In [79]:
show_image(l_channel)

In [80]:
show_image(s_channel)

In [14]:
gradx = abs_sobel_thresh(l_channel, sobel_kernel=ksize, orient='x', thresh=(20, 100))
grady = abs_sobel_thresh(l_channel, sobel_kernel=ksize, orient='y', thresh=(20, 100))
mag_binary = mag_thresh(l_channel, sobel_kernel=ksize, mag_thresh=(100, 255))
dir_binary = dir_threshold(l_channel, sobel_kernel=15, thresh=(np.pi//3, 1.5))

In [15]:
combined = np.zeros_like(dir_binary)
combined[((gradx == 255) & (grady == 255)) | ((mag_binary == 255) & (dir_binary == 255))] = 255

In [83]:
show_image(gradx)

In [84]:
show_image(grady)

In [85]:
show_image(mag_binary)

In [86]:
show_image(dir_binary)

In [87]:
show_image(combined)

In [16]:
# Edit this function to create your own pipeline.
def color_pipeline(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
    
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    s_channel = hls[:,:,2]
    l_channel = hls[:,:,1]
    h_channel = hls[:,:,0]
    
    # Sobel x
    sobelx = cv2.Sobel(s_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Sobel x with l_channel for shadow in the road
    sobelx_l = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0)
    abs_sobelx_l = np.absolute(sobelx_l)
    scaled_sobel_l = np.uint8(255*abs_sobelx_l/np.max(abs_sobelx_l))
    
    # sobelx in s channel
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 255

    # sobelx in l channel
    sxbinary_l = np.zeros_like(scaled_sobel_l)
    sxbinary_l[(scaled_sobel_l >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 255
    
    
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 255

    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) 


    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 255) | (sxbinary == 255) | (sxbinary_l == 255)] = 255

    return sxbinary_l

In [17]:
combined_binary  = color_pipeline(warped)

In [18]:
show_image(combined_binary)

In [182]:
#plt.imshow(combined_binary)

In [19]:
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint]) # take the peak value in the left
    #print("Leftx base value: ", leftx_base)
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint  # take the peak value in the right
    #print("Rightx base value: ", rightx_base)
    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    #print("Nonzero matrix", nonzero)
    nonzeroy = np.array(nonzero[0]) # y value
    nonzerox = np.array(nonzero[1]) # x value
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
    #    print("win_y_low value : ", win_y_low)
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin

        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0, 255, 0), 3)
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0, 255, 0), 3)

        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]

        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)

        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img

In [20]:
leftx, lefty, rightx, righty, out_img = find_lane_pixels(combined_binary)

In [21]:
def fit_poly(img_shape, leftx, lefty, rightx, righty):
     ### TO-DO: Fit a second order polynomial to each with np.polyfit() ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    return left_fitx, right_fitx, ploty

In [22]:
left_fit = np.polyfit(lefty, leftx, 2)
right_fit = np.polyfit(righty, rightx, 2)

In [23]:
def search_around_poly(binary_warped, left_fit, right_fit):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) &
                      (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) &
                      (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)

    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin,
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin,
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)

    # Plot the polynomial lines onto the image
    left_pts = np.vstack((left_fitx, ploty)).astype(np.int32).T
    right_pts = np.vstack((right_fitx, ploty)).astype(np.int32).T
    cv2.polylines(result, [left_pts], False, (0, 255, 255), 10)
    cv2.polylines(result, [right_pts], False, (0, 255, 255), 10)
    ## End visualization steps ##

    return result, left_fitx, right_fitx, ploty



In [24]:
result, left_fitx, right_fitx, ploty = search_around_poly(combined_binary, left_fit, right_fit)

In [25]:
show_image(result)

In [198]:
def original_img(img, undist, left_fitx, right_fitx, ploty, Minv):
    warp_zero = np.zeros_like(img[:,:,0]).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])

    pts = np.hstack((pts_left, pts_right))
    
    # Polynomail lines onto the image
    cv2.polylines(color_warp, np.int_([pts_left]), False, (0, 255, 255), 10)
    cv2.polylines(color_warp, np.int_([pts_right]), False, (0, 255, 255), 10)
    
    
    
    
    cv2.fillPoly(color_warp, np.int_([pts]), (0, 255, 0))

    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0]))
    
        
    

    result = cv2.addWeighted(undist, 1, newwarp, 0.5, 0)
    return result


In [199]:
original = original_img(result, undist, left_fitx, right_fitx, ploty, Minv)

In [200]:
show_image(original)

In [136]:
def proc_pipeline_video(img, objpoints, imgpoints):

    undist = get_undist_img(img, objpoints, imgpoints)
    warped_img, M, Minv = perspective_transform(undist)
    combined_binary = color_pipeline(warped_img)
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(combined_binary)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    result, left_fitx, right_fitx, ploty = search_around_poly(combined_binary, left_fit, right_fit)

    result = original_img(result, undist, left_fitx, right_fitx, ploty, Minv)
    #show_image(result)
    return result

def proc_pipeline(img_path, objpoints, imgpoints):

    img = cv2.imread(img_path)
    undist = get_undist_img(img, objpoints, imgpoints)
    show_image(undist)
    warped_img, M, Minv = perspective_transform(undist)
    combined_binary = color_pipeline(warped_img)
    show_image(combined_binary)
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(combined_binary)
    show_image(out_img)
    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    result, left_fitx, right_fitx, ploty = search_around_poly(combined_binary, left_fit, right_fit)

    result = original_img(result, undist, left_fitx, right_fitx, ploty, Minv)
    show_image(result)
    

In [137]:
def process_video(video_path):
    video_capture = cv2.VideoCapture(video_path)
    while (video_capture.isOpened()):
        ret, frame = video_capture.read()
        if ret:
            output = proc_pipeline_video(frame, objpoints, imgpoints)
            cv2.imshow('frame',output)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
    # Release everything if job is finished
    video_capture.release()
    cv2.destroyAllWindows()

In [138]:
def write_img_from_video(video_path):
    count = 0
    video_capture = cv2.VideoCapture(video_path)
    while (video_capture.isOpened()):
        ret, frame = video_capture.read()
        if ret:
            cv2.imshow('frame',frame)
            
            cv2.imwrite('test_images_new/' + str(count) + ".jpg", frame)
            video_capture.set(cv2.CAP_PROP_POS_MSEC, (count*1000))
            count += 1
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
    # Release everything if job is finished
    video_capture.release()
    cv2.destroyAllWindows()

In [139]:
process_video("challenge.mp4")

In [37]:
!ls test_images_new/

0.jpg  10.jpg  1.jpg  2.jpg  3.jpg  4.jpg  5.jpg  6.jpg  7.jpg	8.jpg  9.jpg


In [36]:
write_img_from_video("challenge.mp4")